In [1]:
# Importando o pandas
import pandas as pd
import openpyxl
import numpy as np
import sys
import os
import re
from numpy import int64
from time import sleep
from xml.dom.minidom import Attr
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait



In [2]:
git remote add origin https://github.com/brenda-marcondes/analise-knlb.git#Dados Yahoo Finance #Não Alterar Mais
url = "https://br.financas.yahoo.com/quote/"
acao = "BEEF3"
url2 = ".SA?p=BEEF3.SA&.tsrc=fin-srch"
site = url + acao + url2
#print(site)

navegador = webdriver.Chrome()

navegador.get(site)

navegador.maximize_window()


page_content =  navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')

sleep(15)

indices = []
tabela1 = []

descricoes = site.findAll('td', attrs={'class' : 'C($primaryColor)'})

for campo in descricoes:
    descricao = campo.text
    indices.append(descricao)
    
    

info = site.findAll('td', attrs={'class' : 'Ta(end)'})

for numeros in info:
    numero = numeros.text
    tabela1.append(numero)
    

dados = {
    'indice': indices[0:16],
    'tabela1': tabela1[0:16]
}

#print(indices)

df = pd.DataFrame(dados)


beta = float(df.iloc[9,1].replace(",", "."))

#Beta alavancado


navegador.close()



In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
#pegando o nome das pastas

arquivos = 'c:\\Users\\bbbre\\OneDrive\\Documentos\\Python\\CVM-Nova-tentativa-22-07-2023\\Klabin'

# Obtém uma lista de todos os nomes de arquivos e pastas no diretório atual
conteudo = os.listdir(arquivos)

# Filtra apenas os nomes de pastas
pastas = [nome for nome in conteudo if os.path.isdir(os.path.join(arquivos, nome))]

#print(pastas[0])



In [5]:
#Criar emdereços para chamar

lista_enderecos =[]

p = len(pastas)

a = 0

base = 'c:\\Users\\bbbre\\OneDrive\\Documentos\\Python\\CVM-Nova-tentativa-22-07-2023\\Klabin'

while a < p:
    endereço =  base + "\\" + pastas [a]  + '\\DadosDocumento.xlsx'
    a = a + 1
    lista_enderecos.append(endereço)

#print(lista_enderecos)

#Nomes das planilhas para chamar read_excel em massa

prefixo = "Trimestre"
quantidade = p
nomes_gerados = [prefixo + str(i) for i in range(quantidade,0,-1)]


b = 0
docs_excel = []

while b < p:
    doc = nomes_gerados[b]
    #print(doc)
    docs_excel.append(doc)
    b =b +1


#Nomes para chamar a função ExcelFile em massa

excelfile = []

c = 0
while  c < p:
    
    nome = nomes_gerados[c]
    excelfile.append(nome)
    c = c + 1

#Atribuir nome e chamar o caminho para todos
#Transforma todos arquivos para leitura 

d = 0
while d < p:
    docs_excel[d] =pd.read_excel(lista_enderecos[d])
    excelfile[d] = pd.ExcelFile(lista_enderecos[d])
    d =d+1
    


In [6]:
nomes_abas = excelfile[1].sheet_names
nomes_abas = nomes_abas[0:10]
print(nomes_abas)
#excelfile[1].sheet_names


['Composicao Capital', 'DF Ind Ativo', 'DF Ind Passivo', 'DF Ind Resultado Periodo', 'DF Ind Resultado Abrangente', 'DF Ind Fluxo de Caixa', 'DF Ind Valor Adicionado', 'DF Ind DMPL Atual', 'DF Ind DMPL Anterior', 'DF Ind DMPL Penultimo']


In [7]:
novas_abas = []

f=1

while f <  len(nomes_abas):
    #aba_do_dataframe é um dataframe de cada aba de uma planilha
    #Selecionando o dataframe de acordo com o índice
    
    aba_do_dataframe = excelfile[0].parse(nomes_abas[f])
    
    #Mudando o valor de uma coluna específica para concatenar posteriormente
    
    if 'Valor Trimestre Atual' in aba_do_dataframe.columns:
        aba = aba_do_dataframe.rename(columns={'Capital Social Integralizado':'Valor Trimestre Atual','Reservas de Capital, Opções Outorgadas e Ações em Tesouraria':'Valor Exercicio Anterior'})

    else:       
        aba = aba_do_dataframe.rename(columns={'Valor Acumulado Atual Exercicio':'Valor Trimestre Atual','Valor Acumulado Exercicio Anterior':'Valor Exercicio Anterior'})
    
    #extrair parte do dataframe
    
    parte_aba = aba.iloc[:, [0,2]]
    nova_aba = pd.DataFrame(parte_aba)
    novas_abas.append(nova_aba)
    f = f+1

    

In [42]:
numero_acoes = excelfile[1].parse(nomes_abas[0]).iloc[0,3]
print(numero_acoes)

5617892756


In [8]:
prefixo = "Trimestre"
quantidade = p
nomes_gerados = [prefixo + str(i) for i in range(quantidade,0,-1)]


lista_dataframes = []

y = 0 #(y é a planilha)

while y < p:

    novas_abas = []
    
    #Renomeia todas as colunhas das abas iterando o nome delas para selecionar cada uma #parse seleciona
    
    f=1
    
    while f <  len(nomes_abas):
        #aba_do_dataframe é um dataframe de cada aba de uma planilha
        #Selecionando o dataframe de acordo com o índice
    
        aba_do_dataframe = excelfile[y].parse(nomes_abas[f])
    
        #Mudando o valor de uma coluna específica para concatenar posteriormente
    
        if 'Capital Social Integralizado' in aba_do_dataframe.columns:
            aba = aba_do_dataframe.rename(columns={'CodigoConta': 'Codigo Conta', 'DescricaoConta': 'Descricao Conta','Capital Social Integralizado':'Valor Trimestre Atual','Reservas de Capital, Opções Outorgadas e Ações em Tesouraria':'Valor Exercicio Anterior'})

        else:       
            aba = aba_do_dataframe.rename(columns={'Valor Acumulado Atual Exercicio':'Valor Trimestre Atual','Valor Acumulado Exercicio Anterior':'Valor Exercicio Anterior'})
    
        #extrair parte do dataframe
        
        parte_aba = aba.iloc[:, [0,1,2]]
        
        #Limpar dados
        
        #parte_aba['Valor Trimestre Atual'] = parte_aba['Valor Trimestre Atual'].apply(lambda x: x.strip() if isinstance(x, str) else x).apply(lambda x: x.replace(".", "") if isinstance(x,str) else x).apply(lambda x: x.replace(",", ".") if isinstance(x,str) else x)
        
        #Criar o nome para as colunas da aba
        
        novo_nome_colunas = {'Valor Trimestre Atual': nomes_gerados[y]}
        
        #renomear as colunas da aba
        
        parte_aba = parte_aba.rename(columns=novo_nome_colunas)
        
        #Criando um novo Dataframe
        
        nova_aba = pd.DataFrame(parte_aba)
        novas_abas.append(nova_aba)
        f = f+1
        
    df = pd.concat(novas_abas, axis= 0).reset_index(drop=True)
    lista_dataframes.append(df)
    
    y = y+1
    
#formulando índice para próprias colunas

lista_dataframes[1].head(3)

,Codigo Conta,Descricao Conta,Trimestre3,Valor Capital Social
0,1,Ativo Total,38.264.536,NaN
1,1.01,Ativo Circulante,10.369.754,NaN
2,1.01.01,Caixa e Equivalentes de Caixa,3.439.810,NaN


In [9]:
prefixo = "Trimestre"
quantidade = p
nomes_gerados = [prefixo + str(i) for i in range(quantidade,0,-1)]


lista_dataframes = []

y = 0 #(y é a planilha)

while y < p:

    novas_abas = []
    
    #Renomeia todas as colunhas das abas iterando o nome delas para selecionar cada uma #parse seleciona
    
    f=1
    
    while f <  len(nomes_abas):
        #aba_do_dataframe é um dataframe de cada aba de uma planilha
        #Selecionando o dataframe de acordo com o índice
    
        aba_do_dataframe = excelfile[y].parse(nomes_abas[f])
    
        #Mudando o valor de uma coluna específica para concatenar posteriormente
    
        if 'Capital Social Integralizado' in aba_do_dataframe.columns:
            aba = aba_do_dataframe.rename(columns={'CodigoConta': 'Codigo Conta', 'DescricaoConta': 'Descricao Conta','Capital Social Integralizado':'Valor Trimestre Atual','Reservas de Capital, Opções Outorgadas e Ações em Tesouraria':'Valor Exercicio Anterior'})

        else:       
            aba = aba_do_dataframe.rename(columns={'Valor Acumulado Atual Exercicio':'Valor Trimestre Atual','Valor Acumulado Exercicio Anterior':'Valor Exercicio Anterior'})
    
        #extrair parte do dataframe
        
        parte_aba = aba.iloc[:, [0,2]]
        
        #Limpar dados
        
        #parte_aba['Valor Trimestre Atual'] = parte_aba['Valor Trimestre Atual'].apply(lambda x: x.strip() if isinstance(x, str) else x).apply(lambda x: x.replace(".", "") if isinstance(x,str) else x).apply(lambda x: x.replace(",", ".") if isinstance(x,str) else x)
        
        #Criar o nome para as colunas da aba
        
        novo_nome_colunas = {'Valor Trimestre Atual': nomes_gerados[y]}
        
        #renomear as colunas da aba
        
        parte_aba = parte_aba.rename(columns=novo_nome_colunas)
        
        #Criando um novo Dataframe
        
        nova_aba = pd.DataFrame(parte_aba)
        novas_abas.append(nova_aba)
        f = f+1
        
    df = pd.concat(novas_abas, axis= 0).reset_index(drop=True)
    lista_dataframes.append(df)
    
    y = y+1
    
#formulando índice para próprias colunas

lista_dataframes[1].head(5)

,Codigo Conta,Trimestre3,Valor Capital Social
0,1,38.264.536,NaN
1,1.01,10.369.754,NaN
2,1.01.01,3.439.810,NaN
3,1.01.02,2.023.632,NaN
4,1.01.02.01,2.023.632,NaN


In [10]:
#Tirando a coluna "Valor Capital Social"
nv_listadataframes = []

for dataframe in lista_dataframes:
    nv_dataframe = dataframe.drop('Valor Capital Social', axis=1)
    nv_listadataframes.append(nv_dataframe)
    
nv_listadataframes[1].head(2)

,Codigo Conta,Trimestre3
0,1,38.264.536
1,1.01,10.369.754


In [11]:
#Convertendo para float
nv_listadataframes2 = []
#seleciona a coluna pra limpar pontos e virgulas
for dataframe_coluna in nv_listadataframes:
    coluna = dataframe_coluna.columns[1]
    dataframe_coluna[coluna] = dataframe_coluna[coluna].apply(lambda x: x.strip() if isinstance(x, str) else x).apply(lambda x: x.replace(".", "") if isinstance(x,str) else x).apply(lambda x: x.replace(",", ".") if isinstance(x,str) else x).astype(float)
    nv_listadataframes2.append(dataframe_coluna)

nv_listadataframes2[2].head(3)


,Codigo Conta,Trimestre2
0,1,42178841.0
1,1.01,12447522.0
2,1.01.01,4330885.0


In [12]:
j = 0
while j < len(nv_listadataframes2):
    tabela_final = nv_listadataframes2[0].merge(nv_listadataframes2[j], on='Codigo Conta', how='left')
    j = j + 1
tabela_final.head(1)


,Codigo Conta,Trimestre4,Trimestre1
0,1,39182141.0,43601585.0


In [14]:
#Tabela base
tabela_final = nv_listadataframes2[0].merge(nv_listadataframes2[1], on='Codigo Conta', how='left')


#Iteração que adiciona novas colunas

j = 2

while j < len(nv_listadataframes2):
    
    tabela_final = tabela_final.merge(nv_listadataframes2[j], on='Codigo Conta', how='left')
    j = j + 1
    

tabela_final.head(2)


,Codigo Conta,Trimestre4,Trimestre3,Trimestre2,Trimestre1
0,1,39182141.0,38264536.0,42178841.0,43601585.0
1,1.01,12834594.0,10369754.0,12447522.0,11284914.0


In [15]:
tabela_final1 = tabela_final.set_index('Codigo Conta')
tabela_final2 = pd.DataFrame(tabela_final1)
tabela_final2['T0'] = 1
tabela_final2.head(2)

,Trimestre4,Trimestre3,Trimestre2,Trimestre1,T0
Codigo Conta,,,,,
1,39182141.0,38264536.0,42178841.0,43601585.0,1
1.01,12834594.0,10369754.0,12447522.0,11284914.0,1


In [16]:
linha = tabela_final1.loc['1.01.01']
#print(linha)

#vendas = tabela_final1.loc['3.01']

#tabela_final1.loc[vendas, 'Resultado'] = tabela_final1.loc[vendas,'Trimestre4'] - tabela_final1.loc[vendas,'Trimestre3']




In [17]:
# ORIGINAL #### Modelo antigo que era base
# Nomes das variações percentuais

#quantidade = p
#sinal = "-"
#nomes_gerados_var_dfs = [prefixo + str(i) + sinal + prefixo for i in range(quantidade, 0, -1)]
#print(nomes_gerados)


#Variação mês a mês de todas as linhas percentual
#g = 0 #Variação do nome

#while g < p:
#    df_tabela_inicial[nomes_gerados_var_dfs[g]] = (df_tabela_inicial[df_tabela_inicial.columns[g]] - df_tabela_inicial[tabela_final2.columns[(g+1)]])/tabela_final2[tabela_final2.columns[(g+1)]] 
    
#    g= g+1
    

#tabela_final2.head(2)

In [18]:
# Nomes das variações percentuais

quantidade = p
sinal = "-"
nomes_gerados_var_dfs = [prefixo + str(i) + sinal + prefixo for i in range(quantidade, 0, -1)]
print(nomes_gerados)


#Variação mês a mês de todas as linhas percentual
g = 0 #Variação do nome

df_variação_percentual = pd.DataFrame()

while g < p:
    df_variação_percentual[nomes_gerados_var_dfs[g]] = (tabela_final2[tabela_final2.columns[g]] - tabela_final2[tabela_final2.columns[(g+1)]])/tabela_final2[tabela_final2.columns[(g+1)]] 
    
    g= g+1
    

df_variação_percentual.head(2)

['Trimestre4', 'Trimestre3', 'Trimestre2', 'Trimestre1']


,Trimestre4-Trimestre,Trimestre3-Trimestre,Trimestre2-Trimestre,Trimestre1-Trimestre
Codigo Conta,,,,
1,0.023981,-0.092803,-0.032631,43601584.0
1.01,0.237695,-0.166922,0.103023,11284913.0


In [19]:
#Variações cheias - Variações de números inteiros

quantidade = p
sinal = "-"
nomes_gerados_var_dfs = [prefixo + str(i) + sinal + prefixo for i in range(quantidade, 0, -1)]
print(nomes_gerados)


#Variação mês a mês de todas as linhas percentual
g = 0 #Variação do nome

df_variação_inteiro = pd.DataFrame()

while g < p:
    df_variação_inteiro[nomes_gerados_var_dfs[g]] = (tabela_final2[tabela_final2.columns[g]] - tabela_final2[tabela_final2.columns[(g+1)]])
    
    g= g+1
    

df_variação_inteiro.head(2)

['Trimestre4', 'Trimestre3', 'Trimestre2', 'Trimestre1']


,Trimestre4-Trimestre,Trimestre3-Trimestre,Trimestre2-Trimestre,Trimestre1-Trimestre
Codigo Conta,,,,
1,917605.0,-3914305.0,-1422744.0,43601584.0
1.01,2464840.0,-2077768.0,1162608.0,11284913.0


In [20]:
#Nomes tabela inicial refeita, para base

quantidade = p
sinal = "-"
nomes_gerados_var_dfs = [prefixo + str(i) + sinal + prefixo for i in range(quantidade, 0, -1)]
print(nomes_gerados)


#Variação mês a mês de todas as linhas percentual
g = 0 #Variação do nome

df_tabela_inicial = pd.DataFrame()

while g < p:
    df_tabela_inicial[nomes_gerados_var_dfs[g]] = tabela_final2[tabela_final2.columns[g]] 
                                                       
    g= g+1
    

type(df_tabela_inicial.loc['3.01'])

['Trimestre4', 'Trimestre3', 'Trimestre2', 'Trimestre1']


pandas.core.series.Series

In [21]:
pd.options.display.float_format = '{:.4f}'.format

#Investimento em giro
#Receita de vendas de acordo com a regreção das vendas esperadas
variacao_vendas = df_variação_percentual.loc['3.01']
df_tabela_inicial.loc['VAR VENDAS'] = variacao_vendas

receita_projetada_vendas =  (abs(df_variação_inteiro.loc['3.01']) * abs((1 +variacao_vendas))) + df_tabela_inicial.loc['3.01'] 
df_tabela_inicial.loc['RECEITA PROJ VENDAS'] = receita_projetada_vendas

#inves_adicional_giro = 0 #receita_projetada_vendas - df_tabela_inicial.loc['3.01']
#df_tabela_inicial.loc['INVEST ADICIONAL GIRO'] = inves_adicional_giro



In [22]:
#CAPEX

CAPEX = abs(df_variação_inteiro.loc['1.02.01.10'])
df_tabela_inicial.loc['CAPEX'] = CAPEX

df_tabela_inicial.loc['RECEITA'] = df_tabela_inicial.loc['3.01']

#Endividamento

passivo = df_tabela_inicial.loc['2']
df_tabela_inicial.loc['PASSIVO'] = passivo

patrimonio_liquido = df_tabela_inicial.loc['2.03']
df_tabela_inicial.loc['PATRIMÔNIO LÍQUIDO'] = patrimonio_liquido

sum_passivo_e_pl = passivo + patrimonio_liquido
df_tabela_inicial.loc['PASSIVO+PL'] =sum_passivo_e_pl

endividamento_proprio =  patrimonio_liquido / sum_passivo_e_pl
df_tabela_inicial.loc['ENDIVIDAMENTO PRÓPRIO'] = endividamento_proprio

endividamento_terceiros =  passivo / sum_passivo_e_pl
df_tabela_inicial.loc['ENDIVIDAMENTO TERCEIROS'] = endividamento_terceiros

#Ativo Circulante - Passivo Circulante - Capital Circulante líquido

CCL = df_tabela_inicial.loc['1.01'] - df_tabela_inicial.loc['2.01']
df_tabela_inicial.loc['CCL'] = CCL

variação_investimento_em_giro = CCL / df_tabela_inicial.loc['3.01']
df_tabela_inicial.loc['VAR INVEST GIRO'] = variação_investimento_em_giro

investimento_adicional_em_giro = receita_projetada_vendas - df_tabela_inicial.loc['3.01'] #variação_investimento_em_giro * df_tabela_inicial.loc['3.01']
df_tabela_inicial.loc['INVEST ADICIONAL GIRO'] = investimento_adicional_em_giro
#fórmula aparentemente errada



#df_tabela_inicial.iloc[800:1000]


In [23]:
IR = 0.34

#Fazendo

depreciacao = df_tabela_inicial.loc['6.01.01.02']
df_tabela_inicial.loc['DEPRECIAÇÃO']= depreciacao

lucro_bruto = df_tabela_inicial.loc['3.03']
df_tabela_inicial.loc['LUCRO BRUTO']= lucro_bruto

ebitda = lucro_bruto - df_tabela_inicial.loc['3.04']
df_tabela_inicial.loc['EBITDA']= ebitda

ebit = ebitda - depreciacao
df_tabela_inicial.loc['EBIT']= ebit

nopat = ebit * (1-IR)
df_tabela_inicial.loc['NOPAT']= nopat

fluxo_de_caixa_ope = nopat + depreciacao
df_tabela_inicial.loc['FX CX OPERACIONAL']= fluxo_de_caixa_ope

#Verificar se a variação em vendas é positiva ou negativa, se for, é perigoso, o fluxo de caixa vai aumentar, o mesmo vale para o capex

fluxo_cx_disponivel_empresa = fluxo_de_caixa_ope - CAPEX - investimento_adicional_em_giro
df_tabela_inicial.loc['FX CX EMPRESA']= fluxo_cx_disponivel_empresa

novas_dividas = endividamento_terceiros*((CAPEX + investimento_adicional_em_giro) - depreciacao)
df_tabela_inicial.loc['NOVAS DÍVIDAS']= novas_dividas

fluxo_cx_acionista = fluxo_cx_disponivel_empresa - novas_dividas
df_tabela_inicial.loc['FX CX ACIONISTA']= fluxo_cx_acionista

capital_oneroso = df_tabela_inicial.loc['2.02.01'] + df_tabela_inicial.loc['2.01.04']
df_tabela_inicial.loc['CAPITAL ONEROSO']= capital_oneroso

####################

investimento = capital_oneroso + patrimonio_liquido
df_tabela_inicial.loc['INVESTIMENTO']= investimento

ROI = nopat / investimento
df_tabela_inicial.loc['ROI']= ROI

ROE = df_tabela_inicial.loc['3.11']/ df_tabela_inicial.loc['2.03']
df_tabela_inicial.loc['ROE']= ROE

Bnopat =    (CAPEX - depreciacao + variação_investimento_em_giro) / nopat
df_tabela_inicial.loc['Bnopat']= Bnopat

Gnopat = Bnopat * ROE
df_tabela_inicial.loc['Gnopat']= Gnopat

#df_tabela_inicial.loc['Ações'] = numero_acoes

#acoes_integralizadas = df_tabela_inicial.loc['Ações']


In [24]:
#Pressuposto
selic = 0.13 
df_tabela_inicial.loc['SELIC']= selic

beneficio_fiscal = (1-IR)
df_tabela_inicial.loc['BENEFICIO FISCAL']= beneficio_fiscal

#O spread de risco é um indicador que mede o risco de crédito de uma empresa. Ele é calculado pela diferença entre a taxa de juros de um título emitido por uma empresa e a taxa livre de risco

spread_empresa = 0.02
df_tabela_inicial.loc['SPREAD EMPRESA']=spread_empresa

ibov_12meses = 0
df_tabela_inicial.loc['IBOV 12M']=ibov_12meses

#Beta média do setor
#Usaremos o Beta normal

#RiscoPais
risco_pais = 0.07
df_tabela_inicial.loc['RISCO PAIS']=risco_pais

In [30]:
#Ki = Risk Free + Spread de Risco da Empresa (taxa de risco acima do mercado/spread de inadimplência ) – Benefício Fiscal 
#Onde 0.13 = Taxa selic e 0.17 = taxa de risco da empresa

Ki = (selic + (spread_empresa-selic)) * beneficio_fiscal
df_tabela_inicial.loc['Ki']=Ki

print(Ki)

#Outra  forma de calcular
# Custo da Dívida (Ki) = Risk Free + Spread da Empresa + Risco País
#Ratind da empresa no mercado internacional (Ex BB: 1.16%)
#Risk free Mercados globais 

Ki2 = selic + 0.016 + risco_pais
df_tabela_inicial.loc['Ki2']=Ki2

0.013200000000000002


In [31]:
#Ke Custo de capital próprio
#Ke = RiskFree + beta*(RetornoMercado - RiskFree)

Ke = selic + beta*(ibov_12meses-selic)
df_tabela_inicial.loc['Ke']=Ke

#Mercados internacionais #retorno esperado de um investimento

#Rj = Kej = RiskFree + betaj *(RetornoMercado - RiskFree) + prêmio pais
#Betaj = beta média do setor

betaMS = 0.05
df_tabela_inicial.loc['BETA SETORIIAL']=betaMS

Kej = 0.13 + betaMS * (ibov_12meses - selic) + risco_pais
df_tabela_inicial.loc['Ke2']=Kej


In [32]:
#WACC

wacc = (Kej * endividamento_proprio) + (Ki * (1-IR)  * endividamento_terceiros)
df_tabela_inicial.loc['WACC']=wacc

wacc.head()


Trimestre4-Trimestre   0.0311
Trimestre3-Trimestre   0.0427
Trimestre2-Trimestre   0.0381
Trimestre1-Trimestre   0.0403
dtype: float64

In [216]:
#Modelo Fluxo de Caixa para empresa

#Hipóteses para método 
#a)os fluxos de caixa são mantidos constantes na perpetuidade, apresentando crescimento anual nulo (g = o);
#b)os fluxos de caixa crescem indeterminadamente a uma taxa anual constante (>0). Nesse caso, admita que a taxa de crescimento contínua seja de 1,5% a.a.

#Crescimento Nulo (g = 0)

In [217]:
#Modelo Fluxo de Caixa para empresa

#Hipóteses para método 
#a)os fluxos de caixa são mantidos constantes na perpetuidade, apresentando crescimento anual nulo (g = o);
#b)os fluxos de caixa crescem indeterminadamente a uma taxa anual constante (>0). Nesse caso, admita que a taxa de crescimento contínua seja de 1,5% a.a.

#Crescimento Nulo (g = 0)

In [47]:
# O FCDE é descontado pelo custo total de capital (WACC), média ponderada dos custos de todas as fontes de financiamento

Valor_da_empresa_Vo = fluxo_cx_disponivel_empresa / (1+ wacc)
df_tabela_inicial.loc['VALOR EMPRESA']= Valor_da_empresa_Vo

df_tabela_inicial.iloc[800:1000]



,Trimestre4-Trimestre,Trimestre3-Trimestre,Trimestre2-Trimestre
Codigo Conta,,,
5.07,4475481.0000,4475625.0000,4475625.0000
5.07,4475481.0000,4475625.0000,4475625.0000
VAR VENDAS,-0.0522,-0.0959,-0.0479
RECEITA PROJ VENDAS,4507576.7969,4953339.3776,5238705.6539
CAPEX,270820.0000,474936.0000,55773.0000
RECEITA,4284073.0000,4519879.0000,4999318.0000
PASSIVO,39182141.0000,38264536.0000,42178841.0000
PATRIMÔNIO LÍQUIDO,5397998.0000,8615569.0000,7973273.0000
PASSIVO+PL,44580139.0000,46880105.0000,50152114.0000


1
